Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv')

Взглянем на данные и сделаем небольшие преобразования

In [3]:
print('Размер датасета:', insurance.shape)
print('Количество пропущенных значений:', insurance.isnull().sum().sum())
insurance['Возраст'] = insurance['Возраст'].astype('int')
insurance['Зарплата'] = insurance['Зарплата'].astype('int')
insurance.head(10)

Размер датасета: (5000, 5)
Количество пропущенных значений: 0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
5,1,43,41000,2,1
6,1,39,39700,2,0
7,1,25,38600,4,0
8,1,36,49700,1,0
9,1,32,51700,1,0


Отделим вектор целевого признака от матрицы признаков

In [4]:
y = insurance['Страховые выплаты']
X = insurance.drop('Страховые выплаты', axis=1)

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a_1 = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество не изменится

**Обоснование:** Подставим в формулу предсказаний нашу новую матрицу признаков, умноженную на обратимую матрицу P. Получим:

$$ a_2 = XP*((XP)^T * XP)^{-1} * (XP)^T * y $$

Раскрыв скобки по формуле: $$ (AB)^{-1} = B^{-1}A^{-1} $$

Получим

$$ a_2 = XP * (XP)^{-1} * (XP^{T})^{-1} * (XP)^{T} * y$$

Видим, что у нас получилось произведение матрицы на обратную ей же, что даёт нам единичную матрицу. То есть

$$ a_2 = E * E * y $$

Или если до конца упростить, получим

$$ a_2 = y $$

То есть наши предсказания равны вектору ответов

## 3. Алгоритм преобразования

**Алгоритм**

В качестве алгоритма преобразования выберем следующий:

Домножим нашу матрицу признаков размером MxN на **обратимую** и случайно сгенерированную матрицу размером NxN

**Обоснование**

Вектор предсказаний линейной регрессии не изменится при таком преобразовании, что и было доказано в пункте 2

## 4. Проверка алгоритма

Создадим случайную матрицу `P` и сразу же проверим её на обратимость

In [11]:
P = np.random.rand(4,4) # создаем случайную матрицу размером 4х4 на которую будем умножать признаки
check_inv = np.linalg.inv(P) # если матрица необратима - в этой строчке будет ошибка

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

Домножим `x_train` и `x_test` на матрицу `P`

In [13]:
x_train_multiplied, x_test_multiplied = np.dot(x_train, P), np.dot(x_test, P)

In [19]:
def calc_r2_score(train_features, test_features, train_target, test_target):
    model = LinearRegression()
    model.fit(train_features, train_target)
    predictions = model.predict(test_features)
    return r2_score(predictions, test_target)

In [20]:
calc_r2_score(x_train, x_test, y_train, y_test)

-0.3707352077955892

In [21]:
calc_r2_score(x_train_multiplied, x_test_multiplied, y_train, y_test)

-0.3707352077942381

### Вывод

Метрика R2 получилась одинаковой (незначительное расхождение скорее всего возникло из-за операций над числами с плавающей точкой)